In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import re    # for regular expressions
from string import punctuation
from nltk.stem import SnowballStemmer    #if you are brave enough to do stemming
from nltk.corpus import stopwords      #if you want to remove stopwords
from nltk.tokenize import word_tokenize
import string
from collections import Counter

### Data visualization

In [128]:
# Import products

products = pd.read_csv('products_train.csv')
products.head()

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück


In [129]:
# Import sessions

sessions = pd.read_csv('sessions_train.csv')
sessions.head()

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE


In [130]:
# temp_session = sessions.copy()

In [131]:
sessions_eng = sessions[sessions['locale']=='UK']

In [132]:
def clean_items(text):

    text = text[1:-1]
    text = re.findall(r"'([^']*)'", text)

    return text

In [133]:
sessions_eng['prev_items'] = sessions_eng['prev_items'].apply(clean_items)

C:\Users\Dacia Logan\AppData\Local\Temp\ipykernel_12964\932254714.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions_eng['prev_items'] = sessions_eng['prev_items'].apply(clean_items)


In [134]:
interacted_items = list(sessions_eng['prev_items'])
interacted_items

[['B0BFDL54Y7', 'B0BFDR9X13', 'B07J4WF8VH', 'B07Y21LDJX'],
 ['B07FM2GLNQ', 'B07GZW3P4W'],
 ['B0021L95HU', 'B07DDL77RY', 'B07DDL77RY'],
 ['B0B2WSZYL2', 'B000I8XZ7O'],
 ['B00LW1APOC', 'B00LW1APOC', 'B00OI6NQUI', 'B09HLDN8W1'],
 ['B079DXT1JM', 'B079DZRVDF', 'B079FD3MGB'],
 ['B09X37JZ54', 'B0B9GH9597', 'B0B9GHMSXS', 'B074PQTR4W'],
 ['B094VLF9QV', 'B07ZBKD187'],
 ['B0954WV7VV', 'B09VN6XSDW', 'B0B2R1KL5G'],
 ['B0845XT59N', 'B00474K8SY', 'B008BZVCVM'],
 ['B081KHS89M', 'B08W6W1SP1', 'B00D8CXYXC'],
 ['B0B4V1BH4L', 'B09S642Y37', 'B0BDZXD2NC', 'B0BB61MC84'],
 ['B005C3R4WM', 'B00439VO6I', 'B00J5EVMOA', 'B006MW0G5E'],
 ['1509800697',
  '1509800662',
  'B07XLTQQJN',
  'B07XVP3R4F',
  'B07BR8JQWC',
  'B07BR8JQWC',
  'B002GJFLQ6'],
 ['B0B18M5PWY',
  'B095SZXGG5',
  'B095SX1S12',
  'B0BBSVZ7SR',
  'B072BP5SCL',
  'B06XT1BSD9',
  'B0B993L23X',
  'B08DM17RDG',
  'B08DM1BC3F',
  'B09Q6KCGC4',
  'B09Q73K99D',
  'B0017IMOL2'],
 ['B0B15M68SJ', 'B007531X5Q', 'B096KJDMKF', 'B019VXFEQ6'],
 ['B003ZGE1G0',
  'B09

In [135]:
unique_items = list(set(i for j in interacted_items for i in j))
unique_items

['B09BFL6722',
 'B0BGHF2KVT',
 'B002WRICCC',
 'B09BT6YRQG',
 'B004345SP6',
 'B0998VCH5B',
 'B07RYTSNDV',
 'B01MG4DM91',
 'B07NZ6VCZG',
 'B09L5PM418',
 'B09X4QDSDF',
 'B005AE5600',
 'B08XKM7VLM',
 'B07VV9HGJ5',
 'B09M8K4RYQ',
 'B08D3VDBJG',
 'B075S9MYQ9',
 'B0B584VQXS',
 'B00KHT0SUQ',
 'B0BGFHC5NR',
 'B07TMLZ7VQ',
 'B00PC9UZY0',
 'B08DPNG346',
 'B08HM9T6PH',
 'B00JJLZHLI',
 'B07HYSJW56',
 'B000IU3W1G',
 'B09YMK6X5M',
 'B074YKX2MG',
 'B00MGHITSU',
 'B08X282LQN',
 'B08H2B1VWC',
 'B08JL4D3C1',
 'B09B2YRXCG',
 'B07NZX471W',
 'B09H69Z3TQ',
 'B0867W5KY9',
 'B00H9EZEEM',
 'B0B971D75B',
 'B084Z3HGTV',
 'B00Y0QDEJE',
 'B0B4KGGWZC',
 'B08QVKBNXN',
 'B004X9Y4VU',
 'B09NZYC8CP',
 'B0972Q55TJ',
 'B00IL0QT1U',
 'B08FC6HHD7',
 'B083B6R4FW',
 'B018W7URNC',
 'B09PL6QZLF',
 'B0BFP9KV5D',
 'B085NMQT75',
 'B09GYRK5GR',
 'B0BHS8PKPY',
 'B07DGMSLHF',
 'B000NVRDL0',
 'B09ZQPYKLF',
 'B09P9Q6DWJ',
 'B013W32G6W',
 'B01KI4O3H8',
 'B08JVHQMMV',
 'B09MQGBHGJ',
 'B0954SS5SW',
 'B079RLZRRM',
 'B09W191194',
 'B09N3QKZ

In [136]:
len(unique_items)

470148

In [137]:
# Check unique locations

products['locale'].unique()

array(['DE', 'JP', 'UK', 'ES', 'FR', 'IT'], dtype=object)

In [138]:
# Exctract the English products only (UK)

products_eng = products[products['locale']=='UK']

In [139]:
products_eng = products_eng[products_eng['id'].isin(unique_items)]

In [140]:
# from sklearn.model_selection import train_test_split

In [141]:
# products_eng_used, products_eng_unused = train_test_split(products_eng, train_size=0.5, random_state=42)

In [142]:
# Check the number of products in the English set

# products_eng.size

In [143]:
# Exctract the English sessions only (UK)

# sessions_eng = sessions[sessions['locale']=='UK']

In [144]:
# Check the number of sessions in the English set

# sessions_eng.size

We see that we have more items than distinct sessions

In [145]:
# continue data analysis ...

### Clean

In [146]:
import re
import nltk

In [147]:
titles = np.array(products_eng['title'])
titles = " ".join(titles)

In [157]:
words = re.findall(r'\w+', titles)

In [158]:
word_counter = Counter(words)

In [159]:
word_frequencies = np.array(list(word_counter.values()))

In [160]:
word_quartiles = np.quantile(word_frequencies, [0,.45,0.5])
word_quartiles

array([1., 1., 2.])

In [161]:
len(word_frequencies)

176991

In [162]:
word_freq_2_more = [k for k,v in word_counter.items() if float(v) >= 2]

In [163]:
len(word_freq_2_more)

94355

In [164]:
big_count = 0

def clean(text, stem_words=True):
    
#     text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
#     print(text)
#     text = re.sub('^(?!.[a-zA-Z])(?!.\d)[a-zA-Z0-9]+$', "", text)
    text = re.findall(r'\b(?:[a-zA-Z]+|\d+)\b', text)

    new_text = []
    for word in text:
        if word_counter[word] > 1:
            new_text.append(word)
    text = new_text

#     global big_count
#     big_count += 1
#     print(big_count)
    
    text = " ".join(text)
    
    return text

In [165]:
products_eng['title'] = products_eng['title'].apply(clean)

### Model creation

In [166]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

Problems:
- items with no description

Need to do:
- normalize text (make undercase, no italic, no bold)

In [167]:
# Train a model that finds similar items based on the title of the items only
# https://www.datacamp.com/tutorial/recommender-systems-python

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

# #Replace NaN with an empty string
# products_eng['title'] = products_eng['title'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(products_eng['title'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(470148, 62117)

In [168]:
#Array mapping from feature integer indices to feature name

tfidf.get_feature_names_out()[-1000:]

array(['yinengd', 'yinfung', 'ying', 'yingbiao', 'yingcen', 'yingee',
       'yingrace', 'yinke', 'yinlai', 'yinleader', 'yinler', 'yinsan',
       'yintry', 'yinuo', 'yinuoday', 'yinuofa', 'yinva', 'yinyoo',
       'yiouou', 'yipetor', 'yipinyin', 'yippee', 'yiran', 'yironkie',
       'yirsur', 'yisan', 'yiseang', 'yishik', 'yisica', 'yisifeinuo',
       'yisinuoo', 'yisitong', 'yisrlery', 'yissvic', 'yisuya',
       'yitahome', 'yithao', 'yithinc', 'yitla', 'yitriden', 'yius',
       'yiutera', 'yiviyar', 'yiweel', 'yiwentec', 'yiwer', 'yiwoo',
       'yixia', 'yixisi', 'yiya', 'yiyoo', 'yiyuaniji', 'yizan',
       'yizemay', 'yizerel', 'yizhet', 'yizish', 'yizyif', 'yjlx',
       'yjwan', 'yjzq', 'yk', 'yking', 'ykk', 'ylab', 'ylang', 'ylc',
       'yllee', 'ylong', 'ymccool', 'ymcrlux', 'ymdz', 'yme', 'ymenow',
       'ymhpride', 'ymhxcy', 'ymina', 'yming', 'ymo', 'ymond', 'ymsz',
       'ymwalk', 'ymx', 'ymyny', 'ynanimery', 'ynouu', 'ynr', 'ynwa',
       'ynybusi', 'yo', 'yoart',

In [115]:
# Import linear_kernel

from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [116]:
# # https://github.com/AlexanderNixon/Machine-learning-reads/blob/master/Movie-content-based-recommender-using-tf-idf.ipynb

# cosine_sim = cosine_similarity(tfidf_matrix)

In [117]:
# tfidf_matrix = tfidf_matrix.astype('float32')

In [173]:
def get_closest_item(item_id):
    return None

In [150]:
tfidf_matrix[0]

<1x148193 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [151]:
# Compute the cosine similarity matrix

cosine_sim = linear_kernel(tfidf_matrix[0], tfidf_matrix)

In [125]:
cosine_sim.shape

(14692, 14692)

In [126]:
#Construct a reverse map of indices and item titles

indices = pd.Series(products_eng.index, index=products_eng['title']).drop_duplicates()

In [127]:
indices

title
Canny Collar No Pull Dog Head Collar Lead Training Head Harness Dog Collar that stops pulling Easy to fit simple to use kind safe comfortable                                      947223
Make It Real DIY Plush Toys Pom Pom Characters Making Kit Arts and Crafts for Kids Gifts for Girls                                                                                1286748
Hanger 32 Pegs Flexible Underwear Socks Hanging Rack Folding Multifunctional In outdoor Washing Laundry Dryer Grey                                                                1206734
WOWLED 10 X Dimmable LED Capsule Light Bulbs Equivalent Cool White for Chandelier Home Lighting 360 Beam Angle                                                                    1223035
Curly Human Hair Wig Short Bob Brazilian Wigs for Black Women Middle Part Afro Curly Wig 150 Density Real Hair Wigs for Women Natural black                                       1268552
                                                                

For report:

- more than half of words used only once
- a lot of products which are not rented